In [1]:
%cd  ../
!pip install sasha -e . --no-deps

c:\Users\medalix\Desktop\Phd_Dev\sasha.v.0.0.1
Looking in indexes: https://pypi.org/simple, https://pypi.ngc.nvidia.com
Obtaining file:///C:/Users/medalix/Desktop/Phd_Dev/sasha.v.0.0.1
  Installing build dependencies: started
  Installing build dependencies: finished with status 'done'
  Checking if build backend supports build_editable: started
  Checking if build backend supports build_editable: finished with status 'done'
  Getting requirements to build editable: started
  Getting requirements to build editable: finished with status 'done'
  Preparing editable metadata (pyproject.toml): started
  Preparing editable metadata (pyproject.toml): finished with status 'done'
  Building editable for sasha (pyproject.toml): started
  Building editable for sasha (pyproject.toml): finished with status 'done'
  Created wheel for sasha: filename=sasha-0.1.0-0.editable-py3-none-any.whl size=3437 sha256=67151b9243e788c86b2625140bf649dcd789d94631e4a19f87f9a5acb5466d30
  Stored in directory: C:\Use


[notice] A new release of pip is available: 24.0 -> 25.0.1
[notice] To update, run: python.exe -m pip install --upgrade pip


In [ ]:
import  jax.numpy as jnp
import  numpy as np
from    sasha.model.common.spectral import Spectral
from    sasha.config.backend import  Backend, get_cp, convert_array
from    sasha.utils.plotting import plot_spectra
from    sasha.model.lee.lee_model import *
from    sasha.model.radiometry.radiometry import Radiometry
from    sasha.model.design.design_base import Design
from    sasha.config.simulationLoader import simulation_conditions
from    sasha.utils.array_utils import ensure_array_param_dict
from    sasha.utils.colors import compute_rgb_for_parametrization
import  matplotlib.pyplot as plt
import  numpy as np


BACKEND            =  Backend.JAX
cp                 =  get_cp(backend = BACKEND)
wvl                =  convert_array(np.arange(400.0,800.0),BACKEND)
SENSOR_OPTION_MS   = "sentinel_2A" #sentinel_2A, sentinel_2B, hyspex_muosli
SENSOR_OPTION_DC   = "dirac" #sentinel_2A, sentinel_2B, hyspex_muosli

modeling_args  = {
    "aphy_model_option":    "log_bricaud",
    "acdom_model_option":   "log_bricaud",
    "anap_model_option":    "log_bricaud",
    "bb_chl_model_option":  "None",
    "bb_cdom_model_option": "log_bricaud",
    "bb_tsm_model_option":  "log_bricaud",
    "bottom_model_option":  "linear" 
    }

design_instance_dc = Design(model_option= SENSOR_OPTION_DC, wvl=wvl, backend = BACKEND)
lee_model_instance_dc       = LeeModel(wvl=design_instance_dc.wvl,**modeling_args, backend = BACKEND)

# Initialize Radiometry
radiometry_instance_dc  = Radiometry(design=design_instance_dc,
    reflectance_model   = lee_model_instance_dc,  # Assuming the same instance for simplicity
    prop_type="rrsp", backend = BACKEND)


design_instance_sen = Design(model_option= SENSOR_OPTION_MS, wvl=wvl, backend = BACKEND)
lee_model_instance_sen = LeeModel(wvl=design_instance_sen.wvl,**modeling_args, backend = BACKEND)

# Initialize Radiometry
radiometry_instance_sen = Radiometry(design=design_instance_sen, response_matrix = design_instance_sen.response_matrix,
    reflectance_model   = lee_model_instance_sen,  # Assuming the same instance for simplicity
    prop_type="rrsp", backend = BACKEND)

simus         = simulation_conditions['predefined']

# Black theme colors
# custom_colors = {
#     'primary': '#e30513',
#     'secondary': '#ffc103',
#     'accent1': '#0099ff',
#     'accent2': '#f7a941',
#     'accent3': '#51a27e',
#     'accent4': '#702e78',
#     'accent5': '#7b003d',
#     'background': '#2a2a2a',
#     'text': '#ffffff',
#     'grid': '#ffffff'
# }

# article_simus = ['CWBB', 'CWDB', 'TWBB', 'TWDB']

# # Custom colors
# custom_colors = {
#     'primary': '#e30513',
#     'secondary': '#ffc103',
#     'accent1': '#0099ff',
#     'background': '#2a2a2a',
#     'text': '#ffffff',
#     'grid': '#ffffff',
#     'line': '#ffffff'  # Line color in white for contrast
# }


# Custom colors for white background plot
# Custom colors for white background plot
custom_colors = {
    'primary': '#e30513',      # Red
    'secondary': '#ffc103',    # Yellow
    'accent1': '#0099ff',      # Blue
    'accent2': '#f7a941',      # Orange
    'accent3': '#51a27e',      # Green
    'accent4': '#702e78',      # Purple
    'accent5': '#7b003d',      # Dark red
    'background': '#ffffff',   # White background
    'text': '#333333',         # Dark gray text
    'grid': '#999999',         # Darker gray grid for better visibility
    'line': '#000000'          # Black line for contrast
}

article_simus = ['CWBB', 'CWDB', 'TWBB', 'TWDB']

# Reset to a clean style first, then apply customizations
plt.style.use('default')  # Reset to default matplotlib style

fig, axes = plt.subplots(4, 1, figsize=(12, 20), constrained_layout=True)
fig.patch.set_facecolor(custom_colors['background'])

# Set publication-friendly fonts and sizes
plt.rcParams.update({
    'font.family': 'serif',    
    'font.serif': ['Times New Roman', 'DejaVu Serif', 'serif'],
    'axes.labelsize': 12,
    'axes.titlesize': 14,
    'legend.fontsize': 10,
    'xtick.labelsize': 10,
    'ytick.labelsize': 10,
    'axes.unicode_minus': False,
    'text.color': custom_colors['text'],
    'axes.labelcolor': custom_colors['text'],
    'xtick.color': custom_colors['text'],
    'ytick.color': custom_colors['text'],
    'axes.titlecolor': custom_colors['text'],
    'axes.grid': True,
    'grid.color': custom_colors['grid'],
    'grid.linestyle': ':',     # Dotted grid lines are more visible in publication
    'grid.linewidth': 0.8,     # Slightly thicker for visibility
    'grid.alpha': 0.5,         # More opaque grid
})

size_coeff = 1.5
line_styles = ['-', '--', '-.', ':', (0, (3, 5, 1, 5))]
line_widths = [2.5, 2, 1.5, 1, 0.5]


for ax in axes:
    ax.set_facecolor(custom_colors['background'])
    # Make sure grid is explicitly enabled for each axis
    ax.grid(True, which='both', color=custom_colors['grid'], linestyle=':', linewidth=0.8, alpha=0.5)
    
    # Keep spines visible but subtle
    for spine in ax.spines.values():
        spine.set_color(custom_colors['text'])
        spine.set_linewidth(0.8)


for i_simu, ax in enumerate(axes):
    ax.set_facecolor(custom_colors['background'])
    
    for i, depth in enumerate([1, 5., 10., 15., 20.]):
        simu = simus[article_simus[i_simu]]
        # simu_params = ensure_array_param_dict({**simu, **{'z': depth}}, backend=BACKEND)
        simu_params = {**simu, **{'z': depth}}
        simu_params['chl'] = jnp.log(simu_params['chl'])
        simu_params['cdom'] = jnp.log(simu_params['cdom'])
        simu_params['tsm'] = jnp.log(simu_params['tsm'])
        
        sampled_reflectance_dc = np.array(radiometry_instance_dc.sampled_reflectance(**simu_params)).T
        RGB_simu = compute_rgb_for_parametrization(sampled_reflectance_dc, design_instance_dc.wvl)
        sampled_reflectance_sen = np.array(radiometry_instance_sen.sampled_reflectance(**simu_params)).T
        print(simu_params)
        # Plot line in white with varying styles
        ax.plot(design_instance_dc.bands,
                sampled_reflectance_dc,
                label=f'Depth {depth} m',
                linestyle=line_styles[i % len(line_styles)],
                linewidth=line_widths[i % len(line_widths)],
                color=custom_colors['line'])
        
        
        # Keep original RGB fill colors
        ax.fill_between(design_instance_dc.bands, 
                       sampled_reflectance_dc.squeeze(), 
                       alpha=0.8,
                       color=RGB_simu)

    ax.tick_params(axis='both', which='major', labelsize=18, colors=custom_colors['text'])
    ax.set_ylabel('Remote sensing reflectance ' + r'$(sr^{-1})$', 
                 fontsize=10*size_coeff, weight='bold', color=custom_colors['text'])
    ax.set_title(f"{article_simus[i_simu]}: {simu['label']}", 
                fontsize=18*size_coeff, weight='bold', color=custom_colors['text'])
    
    if i_simu == 0:
        leg = ax.legend(title='Depth (m)', fontsize=14*size_coeff, 
                       title_fontsize=14*size_coeff, loc='upper right')
        leg.get_frame().set_facecolor(custom_colors['background'])
        leg.get_frame().set_edgecolor(custom_colors['text'])
        for text in leg.get_texts():
            text.set_color(custom_colors['text'])
        leg.get_title().set_color(custom_colors['text'])
    
    ax.set_ylim(0, 0.07)
    ax.set_xlim(wvl.min(), wvl.max())
    
    # Explicitly set grid on for each subplot with stronger settings
    ax.grid(which='major', color=custom_colors['grid'], linestyle=':', linewidth=0.8, alpha=0.5)
    
    # Make tick marks more visible
    ax.tick_params(axis='both', which='major', direction='out', length=4, width=1, 
                  colors=custom_colors['text'], grid_color=custom_colors['grid'], grid_alpha=0.8)
    
    # Add minor ticks for more precise reading
    ax.minorticks_on()
    ax.tick_params(which='minor', length=2, color=custom_colors['text'])
    ax.grid(which='minor', color=custom_colors['grid'], linestyle=':', linewidth=0.5, alpha=0.3)

# Set x-label only for the bottom subplot
axes[-1].set_xlabel('Wavelength ' + r'$(nm)$', 
                   fontsize=11*size_coeff, weight='bold', color=custom_colors['text'])


plt.tight_layout()

plt.savefig('figures/fig_water_column_parameterisation.svg', bbox_inches='tight', dpi=300)
# For PNG output with higher resolution
plt.savefig('figures/fig_water_column_parameterisation.png', bbox_inches='tight', dpi=300)
plt.show()